# 5.5读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## 5.5.1.**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。


In [2]:
import torch
from torch import nn
import torch.nn.functional as F

x = torch.arange(4)
#jupyter文件的默认储存位置
torch.save(x, 'x-file')
#也可以更改存储数据的位置（按照绝对路径进行存储）
torch.save(x,'D:/BaiduNetdiskDownload/x-filess')

torch.save介绍
https://pytorch.org/docs/stable/generated/torch.save.html?highlight=torch+save#torch.save
torch.save(obj,f,pickle_module=pickle,pickle_protocol=DEFAULT_PROTOCOL,_use_new_zipfile_serialization=True)

    obj：需要保存的对象，可以是整个模型或者是模型参数
    f：保存模型的路径,默认是jupyter中配置文件的地址在cmd中输入jupyter notebook --generate-config查找默认存储地址
    也可以是保存文件的绝对路径+保存文件名，如’/home/qinying/Desktop/modelpara.pth’
    pickle_module：用于清除元数据和对象的模块
    pickle_protocol：可以指定覆盖默认协议
    
 torch.load(f,map_location=None,pickle_module=pickle,**pickle_load_args)
 
    f：导入模型文件
    map_location：指定如何重新映射存储位置的字符串或字典
    pickle_module：用于清除元数据和对象的模块
    pickle_load_args_：选择指定的参数传入
 
————————————————
版权声明：本文为CSDN博主「鬼道2022」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/qq_38406029/article/details/121006163


我们现在可以将存储在文件中的数据读回内存。


In [6]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以[**存储一个张量列表，然后把它们读回内存。**]


#### 列表的简单介绍

https://www.runoob.com/python/python-lists.html

列表是最常用的Python数据类型，它可以作为一个方括号内的逗号分隔值出现。

列表的数据项不需要具有相同的类型

创建一个列表，只要把逗号分隔的不同的数据项使用方括号括起来即可。如下所示：

In [8]:
#1,创建列表
list1 = ['physics', 'chemistry', 1997, 2000]
list2 = [1, 2, 3, 4, 5,6]
list3 = ["a", "b", "c", "d"]
#与字符串的索引一样，列表索引从0开始。列表可以进行截取、组合等。
#访问列表中，访问方式与张量类似
print(list1[0])# list1[0]的值
print(list2[1:5])# list2[1:5]，左闭右开的
#2，更新列表
list = []          ## 空列表
list.append('Google')   ## 使用 append() 添加元素
list.append('Runoob')
print(list)
#3,删除列表
list1 = ['physics', 'chemistry', 1997, 2000]
print(list1)
del list1[2]
print("After deleting value at index 2 : ")
print(list1)

physics
[2, 3, 4, 5]
['Google', 'Runoob']
['physics', 'chemistry', 1997, 2000]
After deleting value at index 2 : 
['physics', 'chemistry', 2000]


In [16]:
y = torch.zeros(4)#长度维4的张量
torch.save([x, y],'x-files')#因为列表中的数据类型可以是任意的,为什么文件中的是乱码，一直解决不了
x2, y2 = torch.load('x-files')
x2, y2,[x,y],type([x,y])
#[x,y]是个列表，列表中的元素是两个张量

(tensor([0, 1, 2, 3]),
 tensor([0., 0., 0., 0.]),
 [tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.])],
 list)

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。


In [17]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 5.5.2.**加载和保存模型参数**

保存单个权重向量（或其他张量）确实有用，
**但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处**。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
需要注意的一个重要细节是，这将**保存模型的参数而不是保存整个模型**。
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，**为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数**。但是也可以保存模型本身
让我们从熟悉的多层感知机开始尝试一下。
## 序列化的简单介绍 
为什么要序列化？
如果光看定义我想你很难一下子理解序列化的意义，那么我们可以从另一个角度来推导出什么是序列化, 那么究竟序列化的目的是什么？

其实序列化最终的目的是为了对象可以跨平台存储，和进行网络传输。而我们进行跨平台存储和网络传输的方式就是IO，而我们的IO支持的数据格式就是字节数组。

因为我们单方面的只把对象转成字节数组还不行，因为没有规则的字节数组我们是没办法把对象的本来面目还原回来的，所以我们必须在把对象转成字节数组的时候就制定一种规则（序列化），那么我们从IO流里面读出数据的时候再以这种规则把对象还原回来（反序列化）。

如果我们要把一栋房子从一个地方运输到另一个地方去，序列化就是我把房子拆成一个个的砖块放到车子里，然后留下一张房子原来结构的图纸，反序列化就是我们把房子运输到了目的地以后，根据图纸把一块块砖头还原成房子原来面目的过程

参考链接：https://zhuanlan.zhihu.com/p/40462507


In [19]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)#隐藏层
        self.output = nn.Linear(256, 10)#输出层

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()#创建对象
X = torch.randn(size=(2, 20))
Y = net(X)#s实例化
#print(Y)

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]

#### load_state_dict介绍
#### 如果需要储存加载模型参数，需要state_dict提取包含整个模型参数的字典，然后torch.save()保存该字典, 用load_state_dict加载状态字典

官网介绍：https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=load_state_dic#torch.nn.Module.load_state_dict

https://zhuanlan.zhihu.com/p/82038049#:~:text=load_state_dict%20%28%29%20

**状态字典**：PyTorch 中，一个模型(torch.nn.Module)的可学习参数(也就是权重和偏置值)是包含在模型参数(model.parameters())中的，一个状态字典就是一个简单的 Python 的字典，其键值对是每个网络层和其对应的参数张量。模型的状态字典只包含带有可学习参数的网络层（比如卷积层、全连接层等）和注册的缓存（batchnorm的 running_mean）。优化器对象(torch.optim)同样也是有一个状态字典，包含的优化器状态的信息以及使用的超参数。

pickle介绍：

http://c.biancheng.net/view/5736.html

state_dict()是一个状态字典，是一个函数，返回一个包含整个模型的参数的字典。https://pytorch.org/docs/stable/_modules/torch/nn/modules/module.html#Module.state_dict

load_state_dict()就是加载状态字典（加载模型参数）

**注意**

load_state_dict() 方法必须传入一个字典对象，而不是对象的保存路径，也就是说必须先反序列化字典对象，然后再调用该方法，也是例子中先采用 torch.load() ，而不是直接 model.load_state_dict(PATH)

### clone.eval()介绍
一般情况下，我们训练过程如下：

1、拿到数据后进行训练，在训练过程中，使用

model.train(）：告诉我们的网络，这个阶段是用来训练的，可以更新参数。

2、训练完成后进行预测，在预测过程中，使用

model.eval() ： 模型中有BatchNormalization和Dropout，在预测时使用model.eval()后会将其关闭以免影响预测结果。因为Dropout只用于训练集，而不用于测试集。也就是说告诉我们的网络，这个阶段是用来测试的。

BatchNormalization的介绍：https://blog.csdn.net/hjimce/article/details/50866313

In [20]:
torch.save(net.state_dict(), 'mlp.params')
print(net.state_dict())#是有序字典（按照字典中元素输入的顺序输出，前面已经介绍），包括了两层的权重和偏执

OrderedDict([('hidden.weight', tensor([[ 0.1866, -0.0135,  0.2142,  ...,  0.1292,  0.0491, -0.2192],
        [-0.1308, -0.2162, -0.0658,  ..., -0.0699,  0.1739, -0.1145],
        [ 0.1701, -0.0260,  0.0592,  ..., -0.1591,  0.0612,  0.1662],
        ...,
        [-0.1003, -0.1171, -0.0762,  ..., -0.0857, -0.1743, -0.1784],
        [ 0.0517, -0.2159,  0.0161,  ...,  0.2083,  0.2080, -0.1831],
        [-0.0070,  0.1586,  0.1661,  ..., -0.1415,  0.1816,  0.0863]])), ('hidden.bias', tensor([ 0.1392,  0.1259, -0.0683, -0.0182,  0.0339, -0.2214, -0.2007, -0.0016,
         0.1681, -0.0398, -0.0566, -0.0519,  0.0840, -0.0537,  0.0404,  0.0542,
         0.2163, -0.1604,  0.1298, -0.0571, -0.1252,  0.2039,  0.0754, -0.0363,
         0.0310,  0.1428, -0.1023,  0.0097, -0.0723, -0.0503, -0.1342, -0.0967,
         0.1474, -0.0920, -0.0256, -0.2226, -0.0263, -0.0098, -0.1264, -0.2233,
        -0.0319,  0.0260,  0.0647,  0.2221,  0.0292, -0.1121,  0.1502,  0.1226,
        -0.1003, -0.1387, -0.0841, -0

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)


In [23]:
clone = MLP()#创建了多个对象
clone.load_state_dict(torch.load('mlp.params'))#将前面储存的字典加载出来，并且作为clone中的输入
clone.eval()#从train模式调整为test模式，不再进行训练更改梯度，因为这里并没有训练梯度所以可以不加该说明

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。


In [22]:
Y_clone = clone(X)
Y_clone == Y#Y=net（X)

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。

## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如说，如果你想在一个新的网络中使用之前网络的前两层，你该怎么做？
1. 如何同时保存网络架构和参数？你会对架构加上什么限制？


[Discussions](https://discuss.d2l.ai/t/1839)
